# Imports

In [ ]:
%env SM_FRAMEWORK=tf.keras

import cv2
import numpy as np
import glob
import os

from sklearn.metrics import jaccard_score


# Set to use GPU

In [ ]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)


# Dice coefficient

In [ ]:
# https://github.com/karolzak/keras-unet/blob/9b7aff5247fff75dc4e2a11ba9c45929b9166d1f/keras_unet/metrics.py

#def dice_coef(y_true, y_pred, smooth=1.):
#    y_true_f = K.flatten(y_true)
#    y_pred_f = K.flatten(y_pred)
#    intersection = K.sum(y_true_f * y_pred_f)
#    return (2. * intersection + smooth) / (
#                K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Personal
def dice_coefficient(y_pred, y_true, k = 1):
    intersection = np.sum(y_pred[y_true == k]) * 2.0
    dice = intersection / (np.sum(y_pred) + np.sum(y_true))

    return dice


In [ ]:
#cv2.IMREAD_COLOR: It specifies to load a color image.
#    Any transparency of image will be neglected. It is the default flag.
#    Alternatively, we can pass integer value 1 for this flag.
#cv2.IMREAD_GRAYSCALE: It specifies to load an image in grayscale mode.
#    Alternatively, we can pass integer value 0 for this flag.
#cv2.IMREAD_UNCHANGED: It specifies to load an image as such including alpha channel.
#    Alternatively, we can pass integer value -1 for this flag.

def img_pr_mask_adjust(img):
    img = img / float(255)
    #img = np.array(img)

    #print(img.shape)
    #img = img[:,:, 0] # imread IMREAD_GRAYSCALE
    #print(img.shape)

    img[img >= 0.5] = 1
    img[img < 1] = 0
    img = img.astype('uint8')

    return img


In [ ]:
def print_write_file(file, msg):
    print(msg, end='')
    f.write(msg)


# Vegetation Index

In [ ]:
vi_names = [["01_VARI", "Visible Atmospherically Resistant Index"],
            ["02_ExG", "Excess Green Index"],
            ["03_ExR", "Excess Red Vegetation Index"],
            ["04_ExB", "Excess Blue Vegetation Index"],
            ["05_EXGR", "Excess Green minus Excess Red"],
            ["06_GRVI", "Green Red Vegetation Index"],
            ["07_MGRVI", "Modified Green Red Vegetation Index"],
            ["08_GLI", "Green Leaf Index"],
            ["09_RGBVI", "Red Green Blue Vegetation Index"],
            ["10_IKAW", "Kawashima Index"]]


# Run

In [ ]:
img_path_mask="/media/sda2/home/j/Downloads/0del/00/iv_dl_tests/Recortes/"
#img_path_mask="E:/Backes/Segmentacao Linha Plantio CNN/Recortes/"

# Result to dataset A, B, C, D and E
#test_result= 01 to 12, A to E

test = 1 #f = [16, 32, 64, 128, 256] # Test 01
#test = 9 #f = [16, 16, 16, 16, 16] # Test 09
#test = 13 #f = [16, 16, 16, 16, 16, 16] # Test 13
#test = 14 #f = [16, 16, 16, 16, 16, 16] # Test 14
#test = 15 #f = [16, 16, 16, 16, 16, 16, 16] # Test 15
#test = 16 #f = [16, 32, 64, 128, 256, 512] # Test 16

base_train = 'E' # A, B, C, D, E
#base_train = 'A' # A, B, C, D, E

#use_vi = False #<--------------------------------
use_vi = True #<--------------------------------

use_RGB = True # Use RGB channels
#use_RGB = False

# 1 - loss nan - need normalize 0 to 1???
# 2 to 10 ok

#VIs = [1]
VIs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

if not use_vi:
    VIs = [1]
    use_RGB = True
print("\nVIs:", VIs)

# <-------------------------------------------------------
for v in VIs:
    VIs_to_use = [v]

# <-------------------------------------------------------
# run_value = [
#         [1, 2, 3, 4, 5],
#         [5, 6, 7, 8, 9, 10],
#         [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#     ]

# for v in run_value:
#     print("v:", v)
#     VIs_to_use = v

# <-------------------------------------------------------

    vi_config = (use_vi, VIs_to_use)
    if vi_config[0]:
        print("\nUsing VI - vi_config[0]:", vi_config[0])
        #print(type(vi_config[1]))

        if use_RGB:
            img_channels = 3 + len(vi_config[1]) # Count of channel in the img: RGB + VIs
        else:
            img_channels = len(vi_config[1]) # Count of channel in the img: VIs

        print("\nVI in use:", vi_config[1])
        list_VIs = ''
        for val in vi_config[1]:
            print(vi_names[val - 1])
            list_VIs += "_" + vi_names[val - 1][0]

        print("\nlist_VIs:", list_VIs)
    else:
        print("\nNot using VI - vi_config[0]:", vi_config[0])
        img_channels = 3 # Count of channel in the img and input U-Net model
    print("\nimg_channels:", img_channels)

    if test == 1:
        f = [16, 32, 64, 128, 256] # Test 01
    elif test == 9:
        f = [16, 16, 16, 16, 16] # Test 09
    elif test == 13:
        f = [16, 16, 16, 16, 16, 16] # Test 13
    elif test == 14:
        f = [16, 16, 16, 16, 16, 16, 16] # Test 14
    elif test == 15:
        f = [16, 16, 16, 16, 16, 16, 16, 16] # Test 15
    elif test == 16:
        f = [16, 32, 64, 128, 256, 512] # Test 16
    else:
        raise Exception("Test: " + str(f) + "- f not configured!")

    if use_RGB and img_channels == 3:
        base_output_path = "results/test_" + str(test) + "_U-Net_without_VI_weights_Base_" + base_train
    else: # img_channels >= 4
        base_output_path = "results/test_" + str(test) + "_U-Net_VI" + list_VIs + "_weights_Base_" + base_train

    print("\nConfig:", base_output_path)
    print("test:", test, "f:", f)
    print("use_RGB:", use_RGB)

    #letras_predict="A"
    letras_predict="ABCDE"

    #!dir
    #os.chdir(base_output_path) # cd folder to error in long file name
    #!dir

    file_name_save = base_output_path + "/result_images/results_sumary.txt"
    f = open(file_name_save, "w")

    for base_predict in letras_predict:
        file_name_full = base_output_path + "/result_images/" + base_output_path.split('/')[1] + \
                        "_predict_Base_" + base_predict +".txt"

#         print("len(file_name_full):", len(file_name_full))
#         print("len(file_name_save):", len(file_name_save))

        #f = open(file_name_full, "w")
        f = open(file_name_save, "a")

        print_write_file(f, "\nfile_name_full: " + repr(file_name_full))

        print_write_file(f, "\nbase_train: " + repr(base_train))
        print_write_file(f, "\n\nbase_predict: " + repr(base_predict))

        diretorio_pr = base_output_path + "/result_images/predict_Base_"+ base_predict + "/"

        #<---- test U-net Backes -------------------------------------------------------------------------
        #file_name_full = "a/b/c/a.txt" #<--------------------------------
        #diretorio_pr = "E:/Backes/Segmentacao Linha Plantio CNN/Teste_01E/result-images/" #<-------------

        print_write_file(f, "\n\ndiretorio_pr: " + repr(diretorio_pr))

        diretorio_mask = img_path_mask + "Base_" + base_predict + "_mask/"
        print_write_file(f, "\ndiretorio_mask: " + repr(diretorio_mask))

        print_write_file(f, "\n\nfile_name: " + repr(file_name_full.split('/')[3]))

        nr_files = len(glob.glob(diretorio_mask +"/*.png"))
        #print("\nnr_files:", nr_files)
        print_write_file(f, "\n\nnr_files: " + repr(nr_files))

        dice_score_all = 0.0
        IOU_all = 0.0
        count = 0
        print("\n")

        for name_mask in sorted(glob.glob(diretorio_mask +"/*.png")):
            #print("name_mask:", name_mask)

            name_mask = name_mask.replace("\\","/") # change "\" to "/"
            name_pr = name_mask.split('/')[-1] # linux
            #name_pr = name_mask.split('\\')[-1] # Windows strange path

            img_mask = cv2.imread(name_mask, cv2.IMREAD_GRAYSCALE)

            #print("name_pr:", name_pr)
            #print("diretorio_pr + name_pr:", diretorio_pr + name_pr)

            img_pr = cv2.imread(diretorio_pr + name_pr, cv2.IMREAD_GRAYSCALE)

            #print("Image predict path:", diretorio_pr + name_pr)
            #print("Image mask path:", name_mask)

            #print("\nimg_pr", img_pr)
            #print("\nimg_mask", img_mask)
    #--------------------------------------------------------------------------------------------------------------------------
    #=== test = 3 - dice_score avg: 0.9644982330072099 - round 2 - 0.96
        #elif test == 3: # best result
            #img_pr = cv2.imread(name_pr)
            #img_mask = cv2.imread(diretorio_mask + name_mask)

            img_mask = img_pr_mask_adjust(img_mask)
            img_pr = img_pr_mask_adjust(img_pr)
    #--------------------------------------------------------------------------------------------------------------------------

            # Test dice equal to 1
            #img_pr = img_mask.copy()

            dice_score = dice_coefficient(img_pr, img_mask, k = 1) # k can be 255 or 1
            #print ("Dice Similarity {} : {}".format(name_mask, dice_score))

            IOU = jaccard_score(img_mask.flatten(), img_pr.flatten(), average = "binary")
            #print ("IOU Similarity {} : {}".format(name_mask, IOU))

            dice_score_all += dice_score
            IOU_all += IOU
            count += 1

            print(count, end=' ') # img print number running

        #print("count:", count)

        #print("\n\nIOU_all:", IOU_all)
        print_write_file(f, "\n\nIOU_all: " + repr(IOU_all))

        #print("IOU avg:", IOU_all/count)
        print_write_file(f, "\nIOU avg: " + repr(IOU_all/count))

        #print("IOU avg round 2:", round(IOU_all/count, 2))
        #print("IOU avg round 4:", round(IOU_all/count, 4))

        #print("\ndice_score_all:", dice_score_all)
        print_write_file(f, "\n\ndice_score_all: " + repr(dice_score_all))

        print_write_file(f, "\ndice_score avg: " + repr(dice_score_all/count) + "\n")
        print("dice_score avg:", dice_score_all/count, "- visual_check")

        #print("dice_score avg round 2:", round(dice_score_all/count, 2))
        #print("dice_score avg round 4:", round(dice_score_all/count, 4))
        #f.close()

    f.close()

print("\nAll done!\n")


# morphological_transf

In [ ]:
#img_path_mask = "/media/sda2/home/j/Downloads/0del/00/iv_dl_tests/Recortes/"
#img_path_mask = "E:/Backes/Segmentacao Linha Plantio CNN/Recortes/"

#diretorio_mask ="/media/sda2/home/j/Downloads/0del/00/iv_dl_tests/Recortes/Base_E_mask"
#base_img_pr = "/media/sda2/home/j/Downloads/0del/00/iv_dl_tests/Teste_01E/result-images/"

#diretorio_mask = "/run/media/j/Files_ex/0_f_mest_bkp/datasets/Recortes/"
base_img_pr = "/run/media/j/Files_ex/0_f_mest_bkp_outros/iv_results/results_E_1/"
base_img_pr = base_img_pr +  "test_1_U-Net_without_VI_weights_Base_E/result_images/"

base_img_pr = "/run/media/j/Files_ex/0_f_mest_bkp_outros/new_04_01_2023/results_L/"
base_img_pr = base_img_pr +  "test_9_U-Net_without_VI_weights_Base_L/result_images/"

print("base_img_pr:", base_img_pr)
print(sorted(glob.glob(base_img_pr + "/*/")))
#exit3

#for folder_path in sorted(glob.glob(base_img_pr + "/*/")):
for folder_path in sorted(glob.glob(base_img_pr + "/MORPH*/*/")):
    diretorio_mask = "/run/media/j/Files_ex/0_f_mest_bkp/datasets/Recortes/"

    ## When the last folder name is the base name
    print("\n\nfolder_path:", folder_path)
    folder_name = folder_path.split('/')[-2]
    base_test_now = folder_name.split('_')[2]
    print("base_test_now: ", base_test_now)

    ## When the last folder name is NOT the base name
    #base_test_now = "L"

    diretorio_mask = diretorio_mask + "/Base_" + base_test_now + "_mask/"
    print("diretorio_mask:", diretorio_mask)
    #continue

    file_name_save = folder_path + "0_results_sumary.txt"
    f = open(file_name_save, "w")
    #f = open(file_name_save, "a")

    print_write_file(f, "\npath: " + repr(folder_path))

    nr_files = len(glob.glob(diretorio_mask +"/*.png"))
    #print("\nnr_files:", nr_files)
    print_write_file(f, "\n\nnr_files: " + repr(nr_files))

    dice_score_all = 0.0
    IOU_all = 0.0
    count = 0
    print("\n")

    diretorio_pr = folder_path

    for name_mask in sorted(glob.glob(diretorio_mask +"/*.png")):
        #print("name_mask:", name_mask)

        name_mask = name_mask.replace("\\","/") # change "\" to "/"
        name_pr = name_mask.split('/')[-1] # linux
        #name_pr = name_mask.split('\\')[-1] # Windows strange path

        img_mask = cv2.imread(name_mask, cv2.IMREAD_GRAYSCALE)

        #print("name_pr:", name_pr)
        #print("diretorio_pr + name_pr:", diretorio_pr + name_pr)

        img_pr = cv2.imread(diretorio_pr + name_pr, cv2.IMREAD_GRAYSCALE)

        #print("Image predict path:", diretorio_pr + name_pr)
        #print("Image mask path:", name_mask)

        #print("\nimg_pr", img_pr)
        #print("\nimg_mask", img_mask
    #--------------------------------------------------------------------------------------------------------------------------
#=== test = 3 - dice_score avg: 0.9644982330072099 - round 2 - 0.96
    #elif test == 3: # best result
        #img_pr = cv2.imread(name_pr)
        #img_mask = cv2.imread(diretorio_mask + name_mask)

        img_mask = img_pr_mask_adjust(img_mask)
        img_pr = img_pr_mask_adjust(img_pr)
    #--------------------------------------------------------------------------------------------------------------------------

        # Test dice equal to 1
        #img_pr = img_mask.copy()

        dice_score = dice_coefficient(img_pr, img_mask, k = 1) # k can be 255 or 1
        #print ("Dice Similarity {} : {}".format(name_mask, dice_score))

        IOU = jaccard_score(img_mask.flatten(), img_pr.flatten(), average = "binary")
        #print ("IOU Similarity {} : {}".format(name_mask, IOU))

        dice_score_all += dice_score
        IOU_all += IOU
        count += 1

        print(count, end=' ') # img print number running

    #print("count:", count)

    #print("\n\nIOU_all:", IOU_all)
    print_write_file(f, "\n\nIOU_all: " + repr(IOU_all))

    #print("IOU avg:", IOU_all/count)
    print_write_file(f, "\nIOU avg: " + repr(IOU_all/count))

    #print("IOU avg round 2:", round(IOU_all/count, 2))
    #print("IOU avg round 4:", round(IOU_all/count, 4))

    #print("\ndice_score_all:", dice_score_all)
    print_write_file(f, "\n\ndice_score_all: " + repr(dice_score_all))

    print_write_file(f, "\ndice_score avg: " + repr(dice_score_all/count) + "\n")
    print("dice_score avg:", dice_score_all/count, "- visual_check")

    #print("dice_score avg round 2:", round(dice_score_all/count, 2))
    #print("dice_score avg round 4:", round(dice_score_all/count, 4))
    #f.close()

    f.close()
    #exit2

print("\nAll done!\n")

# End Run